# SCRAPE EMEDICINE

**INPUTS**:
* *data/00_CIE_emedicine.xlsx*

**OUTPUTS**:
* *data/01_CIE_emedicine_info.xlsx*

**NOTAS**

### Imports

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re #regex
import requests
import time
import random

## CONFIGURATION

### Get data

In [ ]:
webs = pd.read_excel("data/00_CIE_emedicine.xlsx")

## FUNCTIONS

In [ ]:
def trim_text(text):
    return re.sub("\n\n+", "\n\n", text).strip()

def get_page(soup,content):
    try:
        text = soup.find(id=content).find(class_='refsection_content').text
        return trim_text(text)
    except:
        try:
            if content == 'content_b2':
                div_b1 = soup.find(id='content_b1')
                text = div_b1.find_next_sibling("div").find(class_='refsection_content').text
                return trim_text(text)
        except:
            return ""
        return ""

def get_online_info(row):
    try:
        web = row["WEB"]
        r = requests.get(web.replace('overview','clinical')) #Cambia la url al apartado de clinical
        soup = BeautifulSoup(r.text, 'lxml')
        time.sleep(random.randint(5,15))
        return [get_page(soup,'content_b1'),get_page(soup,'content_b2')]
    except:
        return ["",""]

def get_offline_info(row):
    try:
        cie = str(row["CIE"])
        name = "offline_data/" + cie.replace('.',',') + ".txt"
        f = open(name, "r")
        return f.read()
    except:
        return ""
    
def online_down(info):
    return (info[0] == "") & (info[1] == "")

## GET ONLINE/OFFLINE DATA

In [ ]:
history = []
examination = []
global_info = []
for index, row in webs.iterrows():
    info = get_online_info(row)
    history.append(info[0])
    examination.append(info[1])
    if online_down(info):
        info = get_offline_info(row)
        global_info.append(info)
    else:
        global_info.append("")
webs["HISTORY"] = history
webs["EXAMINATION"] = examination
webs["GLOBAL"] = global_info

## SAVE DATA

In [ ]:
webs.to_excel("data/01_CIE_emedicine_info.xlsx", index=False)